In [7]:
!which python3
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from dataset_spectrogram import EEGDataset
from torch.utils.data import random_split
import neptune.new as neptune
from torchinfo import summary
from dataset_spectrogram import load_dataset
import random
import torch.utils.data as data

/usr/bin/python3


In [8]:
# load in the dataset


raw_data_dir = '../data'

trainingNights = 14
testNights = 3

training_nights_list = list(range(trainingNights+1))
print(training_nights_list)

training_nights_list.remove(11) 
print(training_nights_list)

print("\n\nTraining set\n")
#training_set = load_dataset(training_nights_list, raw_data_dir)

print("\nTest set\n")
#test_set = load_dataset([x+trainingNights+1 for x in range(testNights)], raw_data_dir)




# Test another set instead
training_set = load_dataset([11], raw_data_dir)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14]


Training set


Test set

../data/study_1A_mat_simple/S_03/night_4/spectrogram_bad_segments.npy
../data/study_1A_mat_simple/S_03/night_4/spectrogram_good_segments.npy
Memory usage: 736.859655 MB

Lengths:

Good data length: 12292
Bad data length: 12292
Caluculated length: 24583


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')



class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(batch_size,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(128,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.Linear(155, 512),
            nn.ReLU(),
            nn.Linear(512,1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.conv_stack(x)
        return logits

In [ ]:
learning_rate = 1e-5
batch_size = 64


model = NeuralNetwork()
model.to(device)

loss = nn.BCEWithLogitsLoss() 

#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X) 
        pred = pred.reshape(-1)
        pred = pred.to(device)
        yFloat = y.type(torch.FloatTensor).to(device)
        
        loss = loss_fn(pred, yFloat)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Neptune logging
        run["training/batch/loss"].log(loss)
        

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
        if batch % 10000 == 0:
            print(f"Predicted values: \n{pred}")
            print(f"Actual values: \n{yFloat}")
            print(f"Difference: \n{(yFloat-pred)}")


def test_loop(dataloader_test, model, loss_fn, test_set = True):
    size = len(dataloader_test.dataset)
    num_batches = len(dataloader_test)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader_test:
            X = X.to(device)
            y = y.to(device)
            pred = model(X).reshape(-1).to(device) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor).to(device)).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    correct /= size
    
    
    
    if test_set:
        print(f"Test set Error: \n Test Set Accuracy: {(100*correct):>0.5f}%, Avg Test Set loss: {test_loss:>8f} \n")
        
        # Neptune logging
        run["testing/batch/test_loss"].log(test_loss)
        run["testing/batch/test_Acc"].log(100*correct)
    
    else:
        print(f"Training Set Error: \n Training Set Accuracy: {(100*correct):>0.5f}%, Avg Training Set loss: {test_loss:>8f} \n")
        
        # Neptune logging
        run["testing/batch/training_loss"].log(test_loss)
        run["testing/batch/training_Acc"].log(100*correct)
    


In [ ]:
train_dataloader = DataLoader(training_set, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
test_dataloader = DataLoader(test_set, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss


In [ ]:

run = neptune.init(
    project="NTLAB/artifactDetect-ear", 
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTA4NzcxMy1lYmQ2LTQ3NTctYjRhNC02Mzk1NjdjMWM0NmYifQ==", # your credentials
    source_files=["train_model_spectrogram.ipynb", "dataset_spectrogram.py"]
)  # Credentials


run['config/dataset/size'] = trainingNights 
run['config/model'] = type(model).__name__
run['config/modelSummary'] = str(model)
run['config/optimizer'] = type(optimizer).__name__
run['config/batch_size'] = batch_size
run['config/test_night'] = testNights
run['config/learning_rate'] = learning_rate


In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)    
    test_loop(test_dataloader, model, loss)
    test_loop(train_dataloader, model, loss, test_set = False)

print("Done!")

In [ ]:
# Test the model, and inspect the errors
from datetime import datetime
now = datetime.now()

print(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}")

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}") # Save



In [ ]:
run.stop() # Stop the neptune logging run

In [ ]:
for batch, (X, y) in enumerate(train_dataloader):
    if sum(sum(sum(np.isnan(X)))):
        print(batch)
        print("X:   NAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\n")
        print(sum(sum(sum(np.isnan(X)))))
        
    if batch % 1000 == 100:
        print(batch)
    

In [9]:
for index, i in enumerate(training_set):
    if sum(sum(np.isnan(i[0]))):
        #print(index)
        #print(i)
        print(f"X:   NAN FOUND!!! At index: {index}. Amount of nans: {sum(sum(np.isnan(i[0])))}")
        #print(sum(sum(np.isnan(i[0]))))
        
    if index % 50000 == 0:
        print(index)
    

0
X:   NAN FOUND!!! At index: 5938. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5940. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5942. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5944. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5946. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5948. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5950. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5952. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5954. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5956. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5958. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5960. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5962. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5964. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5966. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5968. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5970. Amount of nans: 2520
X:   NAN FOUND!!! At index: 5972. Amount of nans: 2520
X:   NAN